In [103]:
import time
import keras
import keras.models
from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks
import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [104]:

save_path = "/Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/"
model_name = "ir_lstm_classification"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60
# num_epochs = 1


In [105]:

#### define functions ####

def model_cycle():
    inputs = Input(shape=(50, 4, 1))
        
    x = Conv2D(48, kernel_size=(3,4),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # parallel line 1
    fx1 = Conv2D(48, kernel_size=(3,1),
                   activation='relu',
                   padding='same')(x)
    fx1 = BatchNormalization()(fx1)
    fx1 = Dropout(0.2)(fx1)
    fx1 = Conv2D(48, kernel_size=(3,1),
                   activation='relu',
                   padding='same')(fx1)
    fx1 = MaxPooling2D((2,1),padding='same')(fx1)
    fx1 = BatchNormalization()(fx1)
    fx1 = Dropout(0.2)(fx1)
    
    # parallel line 2
    fx2 = Conv2D(48, kernel_size=(11,1),
                   activation='relu',
                   padding='same')(x)
    fx2 = BatchNormalization()(fx2)
    fx2 = Dropout(0.2)(fx2)
    fx2 = Conv2D(48, kernel_size=(21,1),
                   activation='relu',
                   padding='same')(fx2)
    fx2 = MaxPooling2D((2,1),padding='same')(fx2)
    fx2 = BatchNormalization()(fx2)
    fx2 = Dropout(0.2)(fx2)
    
    # Add
    x1 = Concatenate(axis=-3)([fx1, fx2])
    x = Add()([x, x1])
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    x = LSTM(20, return_sequences=False)(x)
    x = Dropout(0.2)(x)

    # outputs = Dense(1, activation='linear')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    network = Model(inputs, outputs)
    # network.compile(optimizer='rmsprop',
    #                 loss='mean_squared_error')
    network.compile(optimizer='rmsprop',
                    loss='binary_crossentropy',
                    metrics=["accuracy"])
    return network


In [106]:
    
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq


In [107]:

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)
 

In [108]:
#### preparing data ####

# data_cerevisiae_nucle = pd.read_csv("/Users/keren/Documents/HumanChecm/Cyclization/rawdata/cycle1.txt",delimiter = ",")
data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle1.txt",delimiter = ",")
X1_cutoff1, X1_cutoff2 = np.quantile(data_cerevisiae_nucle["C0"], [0.25, 0.75])
X1 = []
# Response: 1 if in fourth (highest) quartile, 0 if in first (lowest) quartile
Y1 = []
# for sequence_nt in data_cerevisiae_nucle["Sequence"]:
#     X1.append(dnaOneHot(sequence_nt))
for i in range(data_cerevisiae_nucle.shape[0]):
    if data_cerevisiae_nucle["C0"][i] < X1_cutoff1:
        X1.append(dnaOneHot(data_cerevisiae_nucle["Sequence"][i]))
        Y1.append(0)
    elif data_cerevisiae_nucle["C0"][i] > X1_cutoff2:
        X1.append(dnaOneHot(data_cerevisiae_nucle["Sequence"][i]))
        Y1.append(1)
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],50,4,1))
X1_reverse = np.flip(X1,[1,2])
Y1 = np.array(Y1)
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)



In [109]:
# data_random_library = pd.read_csv("/Users/keren/Documents/HumanChecm/Cyclization/rawdata/cycle3.txt", delimiter=",")
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle3.txt", delimiter=",")
X3_cutoff1, X3_cutoff2 = np.quantile(data_random_library["C0"], [0.25, 0.75])
X3 = []
# Response: 1 if in fourth (highest) quartile, 0 if in first (lowest) quartile
Y3 = []
# for sequence_nt in data_random_library["Sequence"]:
#     X3.append(dnaOneHot(sequence_nt))
for i in range(data_random_library.shape[0]):
    if data_random_library["C0"][i] < X3_cutoff1:
        X3.append(dnaOneHot(data_random_library["Sequence"][i]))
        Y3.append(0)
    elif data_random_library["C0"][i] > X3_cutoff2:
        X3.append(dnaOneHot(data_random_library["Sequence"][i]))
        Y3.append(1)
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],50,4,1))
X3_reverse = np.flip(X3,[1,2])
Y3 = np.array(Y3)
# Y3 = data_random_library["C0"].values.astype(float)



In [110]:
# data_tiling = pd.read_csv("/Users/keren/Documents/HumanChecm/Cyclization/rawdata/cycle5.txt", delimiter=",")
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle5.txt", delimiter=",")
X5_cutoff1, X5_cutoff2 = np.quantile(data_tiling["C0"], [0.25, 0.75])
X5 = []
# Response: 1 if in fourth (highest) quartile, 0 if in first (lowest) quartile
Y5 = []
# for sequence_nt in data_tiling["Sequence"]:
#     X5.append(dnaOneHot(sequence_nt))
for i in range(data_tiling.shape[0]):
    if data_tiling["C0"][i] < X5_cutoff1:
        X5.append(dnaOneHot(data_tiling["Sequence"][i]))
        Y5.append(0)
    elif data_tiling["C0"][i] > X5_cutoff2:
        X5.append(dnaOneHot(data_tiling["Sequence"][i]))
        Y5.append(1)
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
Y5 = np.array(Y5)
# Y5 = data_tiling["C0"].values.astype(float)



In [111]:
# data_chr5 = pd.read_csv("/Users/keren/Documents/HumanChecm/Cyclization/rawdata/cycle6.txt", delimiter=",")
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle6.txt", delimiter=",")
X6_cutoff1, X6_cutoff2 = np.quantile(data_chr5["C0"], [0.25, 0.75])
X6 = []
# Response: 1 if in fourth (highest) quartile, 0 if in first (lowest) quartile
Y6 = []
# for sequence_nt in data_chr5["Sequence"]:
#     X6.append(dnaOneHot(sequence_nt))
for i in range(data_chr5.shape[0]):
    if data_chr5["C0"][i] < X6_cutoff1:
        X6.append(dnaOneHot(data_chr5["Sequence"][i]))
        Y6.append(0)
    elif data_chr5["C0"][i] > X6_cutoff2:
        X6.append(dnaOneHot(data_chr5["Sequence"][i]))
        Y6.append(1)
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],50,4,1))
X6_reverse = np.flip(X6,[1,2])
Y6 = np.array(Y6)
# Y6 = data_chr5["C0"].values.astype(float)



In [112]:
# m1 = np.mean(Y1)
# std1 = np.std(Y1)
# Z1 = (Y1-m1)/std1

# m3 = np.mean(Y3)
# std3 = np.std(Y3)
# Z3 = (Y3-m3)/std3


# m5 = np.mean(Y5)
# std5 = np.std(Y5)
# Z5 = (Y5-m5)/std5


# m6 = np.mean(Y6)
# std6 = np.std(Y6)
# Z6 = (Y6-m6)/std6



In [113]:
#### random

from numpy import mean


VALIDATION_LOSS = []
fold_var = 1
# n = Z3.shape[0]
n = Y3.shape[0]

fits = []
detrend = []
times = []
times2 = []

# for train_index, val_index in kf.split(Z3):
for train_index, val_index in kf.split(Y3):
    training_X = X3[train_index]
    training_X_reverse = X3_reverse[train_index]
    validation_X = X3[val_index]
    validation_X_reverse = X3_reverse[val_index]
    # training_Y = Z3[train_index]
    # validation_Y = Z3[val_index]
    training_Y = Y3[train_index]
    validation_Y = Y3[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_random_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_random_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_random_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)
    
    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_binary = (pred_Y > 0.5).astype(int)
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y_reverse_binary = (pred_Y_reverse > 0.5).astype(int)
    pred_Y = ((pred_Y+pred_Y_reverse)/2 > 0.5).astype(int)
    # reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    acc =  mean(np.equal(pred_Y_binary, pred_Y_reverse_binary).astype(int))

    # detrend_int = reg.intercept_
    # detrend_slope = reg.coef_
    # detrend.append([float(detrend_int), float(detrend_slope)])
    detrend.append(acc)

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
   # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, Y1)),np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)

    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, validation_Y)),np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)

    start_time = time.process_time()
    fit = model.predict(X5)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X5_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, Y5)),np.mean(np.square(fit-Y5)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)

    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, Y6)),np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1


Epoch 1/60
174/176 [============================>.] - ETA: 0s - loss: 0.5180 - accuracy: 0.7376
Epoch 1: val_loss improved from inf to 1.37338, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_1.h5
176/176 [==============================] - 5s 15ms/step - loss: 0.5169 - accuracy: 0.7384 - val_loss: 1.3734 - val_accuracy: 0.5160
Epoch 2/60
174/176 [============================>.] - ETA: 0s - loss: 0.3152 - accuracy: 0.8723
Epoch 2: val_loss did not improve from 1.37338
176/176 [==============================] - 2s 12ms/step - loss: 0.3159 - accuracy: 0.8721 - val_loss: 1.5670 - val_accuracy: 0.5176
Epoch 3/60
171/176 [============================>.] - ETA: 0s - loss: 0.2489 - accuracy: 0.9028
Epoch 3: val_loss improved from 1.37338 to 0.62030, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_1.h5
176/176 [==============================] - 2s 12ms/step - loss: 0.2482 - accuracy: 0.9024 - val_loss: 0.6203 - val_accu

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_1/assets


1288/1288 [==============================] - 4s 3ms/step
Epoch 1/60
172/176 [============================>.] - ETA: 0s - loss: 0.5486 - accuracy: 0.7131
Epoch 1: val_loss improved from inf to 1.10628, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_2.h5
176/176 [==============================] - 4s 13ms/step - loss: 0.5470 - accuracy: 0.7145 - val_loss: 1.1063 - val_accuracy: 0.4888
Epoch 2/60
174/176 [============================>.] - ETA: 0s - loss: 0.3272 - accuracy: 0.8631
Epoch 2: val_loss did not improve from 1.10628
176/176 [==============================] - 2s 11ms/step - loss: 0.3264 - accuracy: 0.8639 - val_loss: 1.2179 - val_accuracy: 0.5096
Epoch 3/60
171/176 [============================>.] - ETA: 0s - loss: 0.2600 - accuracy: 0.8964
Epoch 3: val_loss improved from 1.10628 to 0.72743, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_2.h5
176/176 [==============================] - 2s 11ms/step - loss

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_2/assets


...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_9/assets


1288/1288 [==============================] - 4s 3ms/step
Epoch 1/60
175/176 [============================>.] - ETA: 0s - loss: 0.5241 - accuracy: 0.7339
Epoch 1: val_loss improved from inf to 0.71013, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_10.h5
176/176 [==============================] - 5s 14ms/step - loss: 0.5235 - accuracy: 0.7345 - val_loss: 0.7101 - val_accuracy: 0.4864
Epoch 2/60
172/176 [============================>.] - ETA: 0s - loss: 0.3135 - accuracy: 0.8712
Epoch 2: val_loss improved from 0.71013 to 0.35503, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_10.h5
176/176 [==============================] - 2s 14ms/step - loss: 0.3126 - accuracy: 0.8719 - val_loss: 0.3550 - val_accuracy: 0.8828
Epoch 3/60
176/176 [==============================] - ETA: 0s - loss: 0.2500 - accuracy: 0.9006
Epoch 3: val_loss improved from 0.35503 to 0.26958, saving model to /Users/Brody1/Documents/Northwestern/Ji

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_random_10/assets


1288/1288 [==============================] - 4s 3ms/step


In [114]:
#FIXME

detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_random.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_random.txt", index = False)

with open(save_path +model_name+"_time_random.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')
        
with open(save_path +model_name+"_pred_time_random.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')
        


In [ ]:
#FIXME

#### chrv

VALIDATION_LOSS = []
fold_var = 1
n = Z6.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Z6):
    training_X = X6[train_index]
    training_X_reverse = X6_reverse[train_index]
    validation_X = X6[val_index]
    validation_X_reverse = X6_reverse[val_index]
    training_Y = Z6[train_index]
    validation_Y = Z6[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_chrv_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_chrv_"+str(fold_var)+".h5")
    model.save(save_path+ model_name+"_chrv_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X5)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X5_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    


In [ ]:
detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_chrv.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_chrv.txt", index = False)

with open(save_path +model_name+"_time_chrv.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_chrv.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')
        


In [ ]:
#FIXME

#### CN

VALIDATION_LOSS = []
fold_var = 1
n = Z1.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Z1):
    training_X = X1[train_index]
    training_X_reverse = X1_reverse[train_index]
    validation_X = X1[val_index]
    validation_X_reverse = X1_reverse[val_index]
    training_Y = Z1[train_index]
    validation_Y = Z1[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_CN_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_CN_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_CN_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X5)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X5_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    


In [ ]:
#FIXME

detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_CN.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_CN.txt", index = False)

with open(save_path +model_name+"_time_CN.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')
with open(save_path +model_name+"_pred_time_CN.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')



In [117]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
# n = Z5.shape[0]
n = Y5.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Y5):
    training_X = X5[train_index]
    training_X_reverse = X5_reverse[train_index]
    validation_X = X5[val_index]
    validation_X_reverse = X5_reverse[val_index]
    training_Y = Y5[train_index]
    validation_Y = Y5[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_tiling_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_tiling_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_tiling_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_binary = (pred_Y > 0.5).astype(int)
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y_reverse_binary = (pred_Y_reverse > 0.5).astype(int)
    pred_Y = ((pred_Y+pred_Y_reverse)/2 > 0.5).astype(int)
    # reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    acc =  mean(np.equal(pred_Y_binary, pred_Y_reverse_binary).astype(int))
    
    # detrend_int = reg.intercept_
    # detrend_slope = reg.coef_
    # detrend.append([float(detrend_int), float(detrend_slope)])
    detrend.append(acc)

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary).astype(int))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp = [mean(np.equal(fit, Y1)), np.mean(np.square(fit-Y1)), np.mean(fit), np.std(fit), reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, Y3)),np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, validation_Y)),np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_binary = (fit > 0.5).astype(int)
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    fit_reverse_binary = (fit_reverse > 0.5).astype(int)
    # reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    reverse_acc = mean(np.equal(fit_binary, fit_reverse_binary))
    fit = ((fit + fit_reverse)/2 > 0.5).astype(int)
    fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit),reverse_corr]
    fit_tmp =[mean(np.equal(fit, Y6)),np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_acc]
    fits.append(fit_tmp)
    # fit = detrend_int + fit * detrend_slope
    # fit = fit.flatten()
    # fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    # fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    


Epoch 1/60
1156/1159 [============================>.] - ETA: 0s - loss: 0.3106 - accuracy: 0.8691
Epoch 1: val_loss improved from inf to 0.16149, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_1.h5
1159/1159 [==============================] - 16s 12ms/step - loss: 0.3104 - accuracy: 0.8691 - val_loss: 0.1615 - val_accuracy: 0.9403
Epoch 2/60
1158/1159 [============================>.] - ETA: 0s - loss: 0.2093 - accuracy: 0.9201
Epoch 2: val_loss improved from 0.16149 to 0.14242, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_1.h5
1159/1159 [==============================] - 14s 12ms/step - loss: 0.2093 - accuracy: 0.9201 - val_loss: 0.1424 - val_accuracy: 0.9468
Epoch 3/60
1156/1159 [============================>.] - ETA: 0s - loss: 0.1844 - accuracy: 0.9311
Epoch 3: val_loss improved from 0.14242 to 0.12833, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_1.h5
1159/1159 [

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_1/assets


1288/1288 [==============================] - 4s 3ms/step
Epoch 1/60
1156/1159 [============================>.] - ETA: 0s - loss: 0.3065 - accuracy: 0.8701
Epoch 1: val_loss improved from inf to 0.16362, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_2.h5
1159/1159 [==============================] - 15s 11ms/step - loss: 0.3065 - accuracy: 0.8701 - val_loss: 0.1636 - val_accuracy: 0.9420
Epoch 2/60
1155/1159 [============================>.] - ETA: 0s - loss: 0.2037 - accuracy: 0.9216
Epoch 2: val_loss improved from 0.16362 to 0.13383, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_2.h5
1159/1159 [==============================] - 12s 10ms/step - loss: 0.2037 - accuracy: 0.9217 - val_loss: 0.1338 - val_accuracy: 0.9522
Epoch 3/60
1157/1159 [============================>.] - ETA: 0s - loss: 0.1814 - accuracy: 0.9314
Epoch 3: val_loss improved from 0.13383 to 0.10786, saving model to /Users/Brody1/Documents/North

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_2/assets


...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_9/assets


1288/1288 [==============================] - 4s 3ms/step
Epoch 1/60
1159/1159 [==============================] - ETA: 0s - loss: 0.3074 - accuracy: 0.8688
Epoch 1: val_loss improved from inf to 0.17576, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_10.h5
1159/1159 [==============================] - 17s 13ms/step - loss: 0.3074 - accuracy: 0.8688 - val_loss: 0.1758 - val_accuracy: 0.9393
Epoch 2/60
1157/1159 [============================>.] - ETA: 0s - loss: 0.2025 - accuracy: 0.9243
Epoch 2: val_loss improved from 0.17576 to 0.14500, saving model to /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_10.h5
1159/1159 [==============================] - 14s 12ms/step - loss: 0.2025 - accuracy: 0.9243 - val_loss: 0.1450 - val_accuracy: 0.9492
Epoch 3/60
1157/1159 [============================>.] - ETA: 0s - loss: 0.1802 - accuracy: 0.9328
Epoch 3: val_loss improved from 0.14500 to 0.12146, saving model to /Users/Brody1/Documents/Nor

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/Jiping/benchmarks/ir_lstm_tiling_10/assets


1288/1288 [==============================] - 5s 4ms/step


In [118]:
#FIXME

detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_tiling.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+ "_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_tiling.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')
